## How to use mutiples GPUs 
 - using `DataParallel`

In [1]:
## Run a Model using GPU
device = torch.device("cuda:0")
model.to(device)

NameError: name 'torch' is not defined

In [ ]:
## Copy tensor to GPU
mytensor = my_tensor.to(device)

calling `mytensor = my_tensor.to(device)` return a new copy on GPU
- instard on rewriting it `my_tensor`
- need to assign it to a new variable

By default, pytorch will only run the model on one GPU
can make it run on multi GPUs using `DataParallel`

In [ ]:
model = nn.DataParallel(model)

In [37]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [38]:
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [40]:
## Dummy Dataset
class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

In [41]:
rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

In [42]:
class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size, hidePrint=False):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        self.hidePrint = hidePrint

    def forward(self, input):
        output = self.fc(input)
        if not self.hidePrint:
            print("\tIn Model: input size", input.size(),
              "output size", output.size())

        return output

In [43]:
## Making sure have multiple GPUs before doing DataParallel
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = nn.DataParallel(model)
else:
  print('Using: ', device)
model.to(device)

Using:  cuda:0


Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

In [44]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])


## GPUs Count
- when using 1 GPU, the output size will be [30,2]
- but when using more GPUs, the output will be smaller, split to multiple GPUs


- DataParallel splits your data automatically 
- and sends job orders to multiple models on several GPUs
- after each model finishes their job
- DataParaller collects and merges the results before returning to you


In [48]:
import time

def testSpeed(gpu = False):
    device = torch.device("cpu")
    
    startFinal = time.time()
    model = Model(input_size, output_size, hidePrint=True)
    
    if gpu:
        if torch.cuda.is_available():
            device = torch.device("cuda:0")
        if torch.cuda.device_count() > 1:
          print("Let's use", torch.cuda.device_count(), "GPUs!")
          # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
          model = nn.DataParallel(model)
        else:
          print('Using: ', device)
        model.to(device)
    else:
        print('Using: CPU')

    for data in rand_loader:
        start = time.time()
        input = data.to(device)
        output = model(input)
    #     print("Outside: input size", input.size(),
    #           "output_size", output.size())
        end = time.time()
        print(end-start)
    endFinal = time.time()
    print('Total: ',endFinal-startFinal)

In [49]:
testSpeed(gpu=True)

Using:  cuda:0
0.00025844573974609375
0.00046706199645996094
0.00018739700317382812
0.00014019012451171875
Total:  0.0046732425689697266


In [50]:
testSpeed(gpu=False)

Using: CPU
0.0065059661865234375
0.00020360946655273438
0.00016427040100097656
0.00027108192443847656
Total:  0.011322736740112305
